In [1]:
import os
import torch
import numpy as np
import pandas as pd

from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets

In [ ]:
batch_size = 100
training_data = datasets.FashionMNIST(root = "data", train = True, transform = transforms.ToTensor(), download = True)
test_data = datasets.FashionMNIST(root = "data", train = False, transform = transforms.ToTensor(), download = True)

train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

In [6]:
sequence_len = 28
input_len = 28
hidden_size = 128
num_layers = 2
num_classes = 10
Learning_rate = 0.01
num_epochs = 5

In [9]:
class LSTM(nn.Module):
  def __init__(self, input_len, hidden_size, num_layers, num_classes):
    super(LSTM, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_len, hidden_size, num_layers, batch_first = True)
    self.output_layer = nn.Linear(hidden_size, num_classes)

  def forward(self, X):
    hidden_states = torch.zeros(self.num_layers, X.size(0), self.hidden_size)
    cell_states = torch.zeros(self.num_layers, X.size(0), self.hidden_size)
    out, _ = self.lstm(X, (hidden_states, cell_states))
    out = self.output_layer(out[:, -1, :])
    return out

In [10]:
model = LSTM(input_len, hidden_size, num_layers, num_classes)
print(model)

LSTM(
  (lstm): LSTM(28, 128, num_layers=2, batch_first=True)
  (output_layer): Linear(in_features=128, out_features=10, bias=True)
)


In [15]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = Learning_rate)

In [21]:
def train(num_epochs, model, train_dataloader, loss_func):
  total_steps = len(train_dataloader)

  for epoch in range(num_epochs):
    for batch, (image, labels) in enumerate(train_dataloader):
      images = image.reshape(-1, sequence_len, input_len)

      output = model(images)
      loss = loss_func(output, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

In [22]:
train(num_epochs, model, train_dataloader, loss_func)

In [23]:
test_images, test_labels = next(iter(test_dataloader))
test_labels

tensor([9, 2, 1, 1, 6, 1, 4, 6, 5, 7, 4, 5, 7, 3, 4, 1, 2, 4, 8, 0, 2, 5, 7, 9,
        1, 4, 6, 0, 9, 3, 8, 8, 3, 3, 8, 0, 7, 5, 7, 9, 6, 1, 3, 7, 6, 7, 2, 1,
        2, 2, 4, 4, 5, 8, 2, 2, 8, 4, 8, 0, 7, 7, 8, 5, 1, 1, 2, 3, 9, 8, 7, 0,
        2, 6, 2, 3, 1, 2, 8, 4, 1, 8, 5, 9, 5, 0, 3, 2, 0, 6, 5, 3, 6, 7, 1, 8,
        0, 1, 4, 2])

In [24]:
test_output = model(test_images.view(-1, 28, 28))

In [25]:
predicted = torch.max(test_output, 1)[1]
predicted

tensor([7, 2, 1, 1, 4, 1, 5, 2, 7, 7, 2, 7, 7, 1, 4, 1, 9, 4, 9, 0, 4, 7, 7, 7,
        1, 9, 4, 0, 9, 0, 4, 0, 1, 3, 4, 1, 7, 7, 7, 2, 0, 1, 0, 9, 4, 7, 4, 1,
        9, 4, 4, 9, 7, 4, 4, 4, 9, 2, 9, 1, 7, 7, 2, 7, 1, 1, 1, 1, 7, 4, 7, 1,
        4, 4, 4, 1, 1, 2, 2, 2, 1, 4, 7, 2, 7, 0, 3, 4, 0, 4, 7, 1, 4, 7, 1, 9,
        1, 1, 4, 2])